### Import Libraries

In [52]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [53]:
titanic_data = pd.read_csv('Titanic-Dataset.csv')
titanic_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [54]:
titanic_data.describe().T

,count,mean,std,min,25%,50%,75%,max
PassengerId,891.0,446.000000,257.353842,1.00,223.5000,446.0000,668.5,891.0000
Survived,891.0,0.383838,0.486592,0.00,0.0000,0.0000,1.0,1.0000
Pclass,891.0,2.308642,0.836071,1.00,2.0000,3.0000,3.0,3.0000
Age,714.0,29.699118,14.526497,0.42,20.1250,28.0000,38.0,80.0000
SibSp,891.0,0.523008,1.102743,0.00,0.0000,0.0000,1.0,8.0000
Parch,891.0,0.381594,0.806057,0.00,0.0000,0.0000,0.0,6.0000
Fare,891.0,32.204208,49.693429,0.00,7.9104,14.4542,31.0,512.3292


In [56]:
n_data = titanic_data.dropna()

In [ ]:
n_data.describe().T

### Separate input and output features

In [ ]:
x = n_data.drop(columns=['Survived'])
y = n_data['Survived']

In [ ]:
from category_encoders import BinaryEncoder
categorical_features = ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']
encoder = BinaryEncoder()
x_encoded = encoder.fit_transform(x[categorical_features])
x = x.drop(categorical_features, axis=1)
x = x.reset_index(drop=True)
x_encoded = x_encoded.reset_index(drop=True)
x = pd.concat([x, x_encoded], axis=1)

In [ ]:
from imblearn.over_sampling import SMOTE
smote = SMOTE()
x, y = smote.fit_resample(x, y)

In [ ]:
y.value_counts()

Survived
1    123
0    123
Name: count, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= .25, random_state=42)

In [ ]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz
import graphviz

from joblib import dump, load

dcs = DecisionTreeClassifier()
dcs.fit(x_train, y_train)
export_graphviz(dcs, out_file='tree.dot', filled=True, feature_names=x_train.columns)

with open("tree.dot") as f:
    dot_graph = f.read()
graphviz.Source(dot_graph) 

# save model
# dump(dcs, "decision_tree_model.joblib")

['decision_tree_model.joblib']

In [ ]:
from sklearn.metrics import accuracy_score, precision_score
dcs_prediction = dcs.predict(x_test)
dcs_accuracy = accuracy_score(dcs_prediction, y_test)
dcs_precision = precision_score(dcs_prediction, y_test)

print(f"Accuracy: {dcs_accuracy*100}")
print(f"Precision: {dcs_precision*100}")

Accuracy: 72.58064516129032
Precision: 73.07692307692307


In [ ]:
loaded_model = load("decision_tree_model.joblib")
loaded_prediction = loaded_model.predict(x_test)